<a href="https://colab.research.google.com/github/mateosuster/pythonungs/blob/master/codigos/mate_financiera/0_Valor_Presente_y_Futuro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conceptos básicos de Matemática Financiera

A continuación se presenta una programación de conceptos básicos de matemática financiera en Python.

Una noción fundamental que está detrás de estos desarrollos es la idea de poder comparar flujos de dinero en distintos momentos del tiempo. Para ello, utilizaremos el **valor presente** y **valor futuro**.

* ¿Cuál es el valor equivalente de $X hoy en el futuro? El **Valor Futuro** utiliza la *capitalización* para calcular cuánto valdrán los flujos efectivo de hoy en algún momento del futuro (o cuál será el valor final de una inversión)

* ¿Cuál es el valor hoy de tener flujos de $X en el futuro? ¿Cuánto valen hoy esas promesas de pago? El **Valor Presente** utiliza el *descuento* para calcular cuánto valen los flujos efectivo hoy (o cuál es el valor que debería invertirse hoy)

<br>

La formulación matemática de estos conceptos es la siguiente:

<br>

$$VF = VP (1+i)^{n}$$
<br>
$$VP = \frac{VF}{ (1+i)^{n}}$$

Donde:
* $VF$ : Valor futuro
* $VP$: Valor presente
* $i$: interés
* $n$: número de períodos

<br>

Para realizar estos cálculos, utilizaremos el [paquete financiero de numpy](https://numpy.org/numpy-financial/latest/), el cual ya tiene cargadas las funciones necesarias.




![picture](https://www.principlesofaccounting.com/wp-content/gallery/Chapter-24/presentvalueordinary.png)

In [ ]:
#instalamos el paquete
!pip install numpy_financial

## Valor Futuro

¿Cuál es el valor futuro de $1000 en tres años con una tasa de interés del 6% anual?

In [ ]:
# importando librerías
import numpy_financial as npf

x = -1000  # deposito (ojo con el signo)
r = .06    # tasa de interes
n = 3      # cantidad de años

# usando la funcion fv de numpy
FV = npf.fv(pv=x, rate=r, nper=n, pmt=0)
FV


Es importante tener en cuenta que, por convención, los signos negativos representan un flujo de cash flow saliente, por ejemplo un depósito, inversión, ahorro o dinero no disponible hoy en día (*cash outflow*). Al contrario, los cobros, ganancias o flujos positivos, quedan con signo positivo (*cash inflow*)

In [ ]:
# controlamos el resultado
-x * (1 + r)**n

In [ ]:
npf.fv?

Repasemos los parámetros de esta función:

* `pv`  es valor presente de nuestra inversión 
* `rate` es la tasa efectiva de interés que nos rendirá la inversión (o anualidad) por cada período.
* `nper`  es el número de períodos. Tener en cuenta que si aquí estamos utilizando como unidad de medida de años, nuestra tasa de interés deberá estar expresada en la misma unidad.
* `pmt` es el valor de los depósitos que vamos a ir invirtiendo en caso de existir una anualidad. El importe tiene que ser ingresado en negativo, dado que refleja una salida de dinero.
* `when` (opcional) indica cuándo se hace efectivo el flujo de dinero. Como sabemos, el resultado puede ser muy distinto cuando se trate, por ejemplo, de un depósito al comienzo o al final de cada período. Por default, la función es igual tiene seteado este parámetro en función de una anualidad ordinaria, lo cual se expresa como {‘end’, 0}, pero se puede cambiar por {‘begin’, 1} para el calculo de la anualidad anticipada. 

Veamos una visualización de algunos resultados

In [ ]:
# Graficando las funciones de valor futuro con interes de 6 y 12 % a 20 años.

def fv_r(num, r):
    return npf.fv(pv=-1000, rate=r, pmt=0, nper=num) 

t = list(range(0, 21)) #lista de valores de 0 a 20


In [ ]:
import matplotlib.pyplot as plt

plt.suptitle("Valor Futuro de $1000")
plt.title("Con tasa de interés de 6 y 12 %")
plt.plot(t, fv_r(num = t, r=.06), label="interes 6 %")
plt.plot(t, fv_r(num = t, r=.12), label="interes 12 %")
plt.legend(loc='upper left')
plt.show()

## Valor Presente

¿Cuál es la inversión inicial que tenemos que hacer hoy para obtener $1000 en tres meses? Asumiendo una tasa del 6% anual 

In [ ]:
fv = -1000  # valor futuro
r = .06   # tasa de interes anual
n = 3     # cantidad de meses

PV = npf.pv(fv=fv, rate= r/12, nper=n, pmt=0)
PV

In [ ]:
#chequeamos el resultado
-fv / ((1 + r/12)**n)

In [ ]:
x = 1

def pv_r(num, r):
    return npf.pv(fv=-x, rate=r, pmt=0, nper=num)    

t = list(range(0, 11)) #lista de valores de 0 a 10


In [ ]:
import matplotlib.pyplot as plt

plt.suptitle("Valor Presente de $1")
plt.title("Con distintas tasas de interés")
plt.plot(t, pv_r(num =t, r = 0), label="0%")
plt.plot(t, pv_r(num =t, r = 0.05 ), label="5%")
plt.plot(t, pv_r(num =t, r = 0.15), label="15%")
plt.plot(t, pv_r(num =t, r = 0.2), label="20%")
plt.xlabel("Tiempo")
plt.legend(loc='best')
plt.show()

Podemos usar el ciclo `for` para emprolijar el código 

In [ ]:
import matplotlib.pyplot as plt

plt.suptitle("Valor Presente de $1")
plt.title("Con distintas tasas de interés")

for i, label in zip([0, 0.05,.15,.2], ["0%","5%","15%","20%" ] ):
  plt.plot(t, pv_r(num =t, r = i), label=label)

plt.legend(loc="best")
plt.xlabel("Tiempo")
plt.show()

## Anualidades

Las **anualidades** son un flujo de efectivo periódico y regular. Se diferencian en *ordinarias* o *anticipadas* según el flujo de efectivo ocurra al final o al inicio del período.

En el caso del valor futuro, la idea de las anualidades es no sólo quedarse con el cálculo simple de cuánto me va a rendir un sólo deposito inicial a fin de un período, sino también poder **calcular el valor futuro de múltiples depósitos** a una misma tasa de interés. 

<br>
<br>


$$VF_{anualidad} = FE\ \   \mathbf{*}\{\frac{(1+i)^n-1}{i}\}$$

donde,
* $FE$ es el flujo de efectivo
<br>
<br>


Por ejemplo, queremos hacer 10 depósitos anuales de $1000 cada uno, los cuales vamos a ir realizando al final de cada año (anualidad ordinaria). ¿Cuál sería en este caso el valor futuro de nuestra anualidad luego del décimo año? Comenzamos con un valor presente de cero y suponemos una tasa de interés del 36%



In [ ]:
# Calculando el valor de la anualidad ordinaria con 36% anual

x = -1000 # valor de depositos
r = .36   # tasa de interes
n = 10    # cantidad de años

FV = npf.fv(pv=0, rate=r, nper=n, pmt=x,  when='end')
FV

In [ ]:
#recordemos que por default, la funcion realiza el cálculo de la anualidad ordinaria (when = 0)
npf.fv(pv=0, rate=r, nper=n, pmt=x)

Si se tratara de una anualidad anticipada, los depósitos se realizarían al inicio del período (`when` = 'begin'), el valor es mayor por las ganancias relativas de interés que vamos teniendo al realizar los depósitos al principio de cada período en lugar de al final (no olvideis una de las grandes máximas fetichistas: *el tiempo es dinero!*).

In [ ]:
# mismo caso pero con la diferencia de que los depositos se realizan al inicio de cada período.
FV = npf.fv(pv=0, rate=r, nper=n, pmt=x, when='begin')
FV

Las anualidades también pueden encontrarse en el **valor presente**. En este caso, inverso al anterior, tenemos una serie de pagos que recibiremos, y el valor presente de la anualidad nos dirá cuál es el valor que tienen hoy esos pagos futuros. 

<br>
<br>


$$VP_{anualidad} = \frac{FE}{i} *[ 1- \frac{1}{(1+i)^n}]$$

donde,
* $FE$ es el flujo de efectivo
<br>
<br>


Así, por ejemplo, si mi tíe en lugar de regalarme USD 10000 dentro de 4 años, decide dar USD 2500 al final de cada año durante 4 años, asumiendo la misma tasa de interés que nos ofrece el banco de 8% anual, el valor presente de esta anualidad se calcula como sigue:

In [ ]:
PV = npf.pv(fv=0, rate=0.08, nper=4, pmt=-2500, when='end')
PV

#  Ejercicios

## Instrumento random 

¿Cuál es el valor presente de un instrumento financiero argentino random que nos promete cinco pagos anuales de $700, si la tasa de interés que rige es del 8% anual

## Valor futuro de algún monto 

¿Cuál es el valor futuro de $70000 luego de 9 meses, existiendo una tasa de interés anual del 38% que se computa mensualmente? Grafique como evoluciona ese valor futuro en muchos períodos el tiempo

## Alquiler

Un gran propietario de departamentos de Palermo, le pide a usted que calcule el valor presente de los pagos de $50000 que le realizará uno de sus inquilinos durante los próximos 37 meses (es un contrato medio extraño). La tasa de interés es del 20% anual
 

## Inversión inicial con ahorros (anualidades)

¿Cuál es el valor presente de una inversión que busca obtener un total de \$15692.93 luego de 10 años? Asumiendo que la tasa de interés es del 5% anual (pero se computa mensualmente) y que todos los meses podemos ahorrar $100 (anualidad)

## Valor futuro con ahorros

¿Cuál es el valor futuro de los ahorros de un jubilade que hoy valen  \$100, luego de depositarlos durante 10 años, al cual le agrega un ahorro adicional de $50 todos los meses? Asuma que la tasa de interés es del 5% anual y se contabiliza mensualmente

## Loop con tasas

Construya un ciclo while aplicado en alguna de las funciones anteriores. Elija alguno de los parámetros sobre los cuales iterar con una lista de valores (de intereses, períodos, montos, valores futuros, presentes, etc.)

## Comparación gráfica de anualidad ordinaria vs anticipada

Compare en un mismo gráfico, el valor futuro de dos anualidades con el mismo monto, tasa y período, pero que se diferencian por ser ordinaria y anticipada